# Rapprochement DPE Audit août 2025

In [2]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
import os
data_path = os.path.join(os.getcwd(), 'data')
print(f"{data_path=}")

data_path='/app/notebooks/rapprochements/DPE/audit/data'


In [3]:
usecols = ['n_audit', 'identifiant_BAN', 'code_postal_BAN', 'code_postal_brut', 'id_RNB']
dtype={'n_audit': 'string', 'identifiant_BAN': 'string', 'code_postal_BAN': 'string', 'code_postal_brut': 'string', 'id_RNB': 'string'}

In [4]:
df_audit = pd.read_csv(os.path.join(data_path, 'audit-opendata.csv'), sep=',', usecols=usecols, dtype=dtype)

In [5]:
df = df_audit

In [6]:
df.shape

(2054519, 5)

In [7]:
df.head()

,n_audit,id_RNB,code_postal_BAN,identifiant_BAN,code_postal_brut
0,A23770111876U,<NA>,77200,77468_0150_00017,77200
1,A24710044341C,<NA>,71960,71371_0005_00796,71960
2,A24430029813G,<NA>,43140,43227_0110_00030,43140
3,A24610161710X,<NA>,61120,61508_0025,61120
4,A24120064359T,<NA>,12200,12300_3085_00074,12200


In [8]:
# create 100 subfiles in the dpe_existant folder, using the number of rows in the dataframe
for i in range(0, 100):
    df.iloc[i*df.shape[0]//100:(i+1)*df.shape[0]//100].to_csv(f'{data_path}/sub_files/dpe-{i}.csv', index=False)

OSError: Cannot save file into a non-existent directory: '/app/notebooks/rapprochements/DPE/audit/data/sub_files'

In [7]:
# le rapprochement

import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from django.db import connection
from concurrent.futures import ThreadPoolExecutor
import numpy as np

def get_rnb_id(row):
    cursor = connection.cursor()

    if row['code_postal_BAN'] != row['code_postal_brut']:
        return []
    
    ban_id = row['identifiant_BAN']

    sql = f"""
            with rnb_ids as (
            select
                rnb_id
            from
                batid_buildingaddressesreadonly bb
            left join batid_building bb2 on
                bb2.id = bb.building_id
            where
                address_id = '{ban_id}'
                and ST_AREA(shape::geography) > 25)
            select
                array_agg(rnb_id)
            from
                rnb_ids;
    """
    cursor.execute(sql)
    result = cursor.fetchone()
    connection.close()
    return result[0] if result[0] is not None else []

def execute(df):
    df_copy = df.copy()
    df_copy['rnb_id_rappro'] = df_copy.apply(get_rnb_id, axis=1)
    return df_copy


def process_sub_file(i):
    print(f"processing file {i}")
    if not os.path.exists(f'{data_path}/sub_files_results/dpe-{i}-result.csv'):
        df_sub_file = pd.read_csv(f'{data_path}/sub_files/dpe-{i}.csv', sep=',')
        max_workers = 20
        dfs = np.array_split(df_sub_file, max_workers)

        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            results_sub_file = executor.map(execute, dfs)
            df_result = pd.concat(results_sub_file)
            df_result.to_csv(f'{data_path}/sub_files_results/dpe-{i}-result.csv', index=False)

for i in range(100):
    process_sub_file(i)

processing file 0
processing file 1
processing file 2
processing file 3
processing file 4
processing file 5
processing file 6
processing file 7
processing file 8
processing file 9
processing file 10
processing file 11
processing file 12
processing file 13
processing file 14
processing file 15
processing file 16
processing file 17
processing file 18
processing file 19
processing file 20
processing file 21
processing file 22
processing file 23
processing file 24
processing file 25
processing file 26
processing file 27
processing file 28
processing file 29
processing file 30
processing file 31
processing file 32
processing file 33
processing file 34
processing file 35
processing file 36
processing file 37
processing file 38
processing file 39
processing file 40
processing file 41
processing file 42
processing file 43
processing file 44
processing file 45
processing file 46
processing file 47
processing file 48
processing file 49
processing file 50
processing file 51
processing file 52
pro

/venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


processing file 86


/venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


processing file 87


...

/venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


processing file 98


/venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


processing file 99


/venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [10]:
# aggregation des sous fichiers de résultats en un seul
dfs_result = []

for i in range(100):
    df_result = pd.read_csv(f'{data_path}/sub_files_results/dpe-{i}-result.csv', usecols=['n_audit', 'rnb_id_rappro'])
    dfs_result.append(df_result)

df_final = pd.concat(dfs_result)
df_final['rnb_id_rappro'] = df_final['rnb_id_rappro'].apply(eval)


In [11]:
# sauvegarde des résultats
df_final.to_csv(f'{data_path}/results_DPE_audit_RNB.csv', index=False)

In [ ]:
df_final = pd.read_csv(f'{data_path}/results_DPE_audit_RNB.csv')

In [ ]:
df_final.shape

In [ ]:
df_final.head()